In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import TimestampType

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_chamado_log_invalidos"

nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"

In [0]:
query = f"""
CREATE OR REPLACE TABLE {nome_tabela_destino} (
    cd_chamado BIGINT COMMENT 'Identificador único do chamado (PK).',
    cd_cliente STRING COMMENT 'Hash SHA256 do identificador do cliente (anonimizado).',
    dh_abertura TIMESTAMP COMMENT 'Data e hora de abertura do chamado.',
    dh_inicio TIMESTAMP COMMENT 'Data e hora de início do atendimento.',
    dh_fim TIMESTAMP COMMENT 'Data e hora de finalização do atendimento.',
    dt_ingestion TIMESTAMP COMMENT 'Data e hora da ingestão do registro.',
    dc_origem STRING COMMENT 'Identificação do sistema ou arquivo fonte dos dados.',
    flag_chamado_valido BOOLEAN COMMENT 'Indica se o ID do chamado é válido.',
    flag_cliente_valido BOOLEAN COMMENT 'Indica se o ID do cliente é válido.',
    flag_timestamps_validos BOOLEAN COMMENT 'Indica se os timestamps foram convertidos com sucesso.',
    flag_qualidade STRING COMMENT 'Status de qualidade: OK ou ERRO.',
    CONSTRAINT pk_chamado_log_invalidos PRIMARY KEY (cd_chamado)
)
COMMENT 'Tabela silver de auditoria - registros de logs de chamados rejeitados por problemas de qualidade';
"""

spark.sql(query)
print(f"✅ Tabela {nome_tabela_destino} criada com sucesso!")